## 7.1 Introduction to the GPT family

In [1]:
from transformers import pipeline, set_seed, GPT2Tokenizer, GPT2LMHeadModel
from torch import tensor, numel
from bertviz import model_view

set_seed(42)

In [2]:
generator = pipeline('text-generation', model='gpt2')

generator("Hello, I'm a language model and I", max_length=30, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model and I don't want to be a computer programmer. As for all those bad languages in JavaScript, I'll never"},
 {'generated_text': "Hello, I'm a language model and I know that's not the right language to build this program of yours. What was wrong with the code?"},
 {'generated_text': "Hello, I'm a language model and I'm trying to work with that much of this, and have this kind of a great conversation about language model"}]

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

'sinan' in tokenizer.get_vocab()

False

In [4]:
tokenizer.convert_ids_to_tokens(tokenizer.encode('Sinan loves a beautiful day'))

['Sin', 'an', 'Ġloves', 'Ġa', 'Ġbeautiful', 'Ġday']

In [5]:
tokenizer.encode('Sinan loves a beautiful day')

[46200, 272, 10408, 257, 4950, 1110]

In [6]:
encoded = tokenizer.encode('Sinan loves a beautiful day', return_tensors='pt')

encoded

tensor([[46200,   272, 10408,   257,  4950,  1110]])

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [82]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [8]:
model.transformer.wte(encoded).shape

torch.Size([1, 6, 768])

In [9]:
model.transformer.wpe(tensor([0, 1, 2, 3, 4, 5]).reshape(1, 6)).shape

torch.Size([1, 6, 768])

In [10]:
initial_input = model.transformer.wte(encoded) + model.transformer.wpe(tensor([0, 1, 2, 3, 4, 5]).reshape(1, 6))

initial_input.shape

torch.Size([1, 6, 768])

In [11]:
initial_input = model.transformer.drop(initial_input)
initial_input

tensor([[[ 0.0107, -0.2453,  0.1275,  ..., -0.1969,  0.0006,  0.1539],
         [-0.1013, -0.0894, -0.0378,  ..., -0.0534, -0.0527,  0.0046],
         [-0.0716, -0.1690,  0.0386,  ..., -0.2034, -0.0197, -0.1113],
         [-0.0509, -0.0682,  0.1526,  ...,  0.0527,  0.0912, -0.0455],
         [ 0.0612,  0.1425,  0.1402,  ...,  0.0964,  0.0510,  0.1474],
         [-0.1283, -0.0632,  0.1287,  ..., -0.0907, -0.0655,  0.1085]]],
       grad_fn=<AddBackward0>)

In [12]:
model.lm_head

Linear(in_features=768, out_features=50257, bias=False)

In [13]:
for module in model.transformer.h:
    initial_input = module(initial_input)[0]
    
initial_input = model.transformer.ln_f(initial_input)

In [14]:
(initial_input == model(encoded, output_hidden_states=True).hidden_states[-1]).all()

tensor(True)

In [16]:
total_params = 0
for param in model.parameters():
    total_params += numel(param)
    
print(f'Number of params: {total_params:,}')

Number of params: 124,439,808


## 7.2 Masked multi-headed attention

In [17]:
import torch
import pandas as pd


In [18]:
phrase = 'My friend was right about this class. It is so fun!'
encoded_phrase = tokenizer(phrase, return_tensors='pt')

response = model(**encoded_phrase, output_attentions=True, output_hidden_states=True)

len(response.attentions)

12

In [84]:
encoded_phrase

{'input_ids': tensor([[3666, 1545,  373,  826,  546,  428, 1398,   13,  632,  318,  523, 1257,
            0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [19]:
response.attentions[-1].shape  # From the final decoder

torch.Size([1, 12, 13, 13])

In [20]:
encoded_phrase['input_ids'].shape

torch.Size([1, 13])

In [21]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])

tokens

['My',
 'Ġfriend',
 'Ġwas',
 'Ġright',
 'Ġabout',
 'Ġthis',
 'Ġclass',
 '.',
 'ĠIt',
 'Ġis',
 'Ġso',
 'Ġfun',
 '!']

In [22]:
# Layer index 9, head 0. Check out the almost 60% attention the token it is giving to the token class
arr = response.attentions[9][0][0]

n_digits = 3

attention_df = pd.DataFrame((torch.round(arr * 10**n_digits) / (10**n_digits)).detach()).applymap(float)

attention_df.columns = tokens
attention_df.index = tokens

attention_df


,My,Ġfriend,Ġwas,Ġright,Ġabout,Ġthis,Ġclass,.,ĠIt,Ġis,Ġso,Ġfun,!
My,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġfriend,0.968,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġwas,0.824,0.145,0.031,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġright,0.979,0.008,0.007,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġabout,0.979,0.008,0.004,0.005,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġthis,0.924,0.031,0.007,0.006,0.016,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġclass,0.946,0.005,0.001,0.001,0.001,0.002,0.044,0.000,0.000,0.000,0.000,0.000,0.000
.,0.691,0.013,0.003,0.003,0.002,0.006,0.269,0.013,0.000,0.000,0.000,0.000,0.000
ĠIt,0.318,0.003,0.003,0.003,0.006,0.018,0.599,0.018,0.032,0.000,0.000,0.000,0.000
Ġis,0.331,0.006,0.002,0.002,0.003,0.018,0.533,0.013,0.062,0.030,0.000,0.000,0.000


In [83]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0]) 
model_view(response.attentions, tokens)

<IPython.core.display.Javascript object>

In [24]:
response.hidden_states[-1].shape

torch.Size([1, 13, 768])

In [25]:
response.logits.shape

torch.Size([1, 13, 50257])

In [26]:
pd.DataFrame(
    zip(tokens, tokenizer.convert_ids_to_tokens(response.logits.argmax(2)[0])), 
    columns=['Sequence up until', 'Next token with highest probability']
)

,Sequence up until,Next token with highest probability
0,My,Ċ
1,Ġfriend,","
2,Ġwas,Ġa
3,Ġright,.
4,Ġabout,Ġthat
5,Ġthis,.
6,Ġclass,.
7,.,ĠI
8,ĠIt,'s
9,Ġis,Ġa


In [86]:
generator(phrase, max_length=20, num_return_sequences=1, do_sample=False)  # greedy search

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My friend was right about this class. It is so fun! I love it! I love the'}]

In [28]:
generator(phrase, max_length=20, num_return_sequences=1, do_sample=True)  # greedy search with sampling

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My friend was right about this class. It is so fun! I wanted to do a different class'}]

## 7.3 Pre-training GPT

In [29]:
from transformers import pipeline, set_seed
from torch import tensor

generator = pipeline('text-generation', model='gpt2', tokenizer=tokenizer)
set_seed(0)

In [30]:
# Bias
generator("The holocaust was", max_length=10, num_return_sequences=10, temperature=0.8, beams=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The holocaust was a cruel and degrading and brutal'},
 {'generated_text': 'The holocaust was created in order to save the'},
 {'generated_text': 'The holocaust was a crime carried out by Jews'},
 {'generated_text': 'The holocaust was perpetrated on the Jews who died'},
 {'generated_text': 'The holocaust was never officially discovered.\n\n'},
 {'generated_text': 'The holocaust was brought to an end on June'},
 {'generated_text': 'The holocaust was an event that happened in the'},
 {'generated_text': 'The holocaust was the product of human sin.'},
 {'generated_text': 'The holocaust was "the most horrible crime in'},
 {'generated_text': 'The holocaust was the worst in history. The'}]

In [31]:
generator("Jewish people are", max_length=10, num_return_sequences=10, temperature=0.8, beams=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Jewish people are not the primary target. They are'},
 {'generated_text': 'Jewish people are not allowed to be on the same'},
 {'generated_text': 'Jewish people are in the U.S. illegally'},
 {'generated_text': 'Jewish people are being persecuted by Muslim extremists."\n'},
 {'generated_text': "Jewish people are not just a minority – they're"},
 {'generated_text': 'Jewish people are a real danger." He said:'},
 {'generated_text': 'Jewish people are not the same as Jews. They'},
 {'generated_text': 'Jewish people are a tiny minority. These are people'},
 {'generated_text': 'Jewish people are called "foreigners" or "'},
 {'generated_text': 'Jewish people are often treated as terrorists. For example'}]

In [32]:
generator("The earth is", max_length=10, num_return_sequences=10, temperature=0.8, beams=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The earth is a giant ball with millions of tiny'},
 {'generated_text': 'The earth is an incredibly thin layer of matter,'},
 {'generated_text': 'The earth is flat, but there are no clouds'},
 {'generated_text': 'The earth is a sphere, an axis, a'},
 {'generated_text': 'The earth is not an island, but a region'},
 {'generated_text': 'The earth is dying due to this event!" The'},
 {'generated_text': 'The earth is flat, but it bends.\n'},
 {'generated_text': 'The earth is flat. I just need to get'},
 {'generated_text': 'The earth is thin but it is thick; we'},
 {'generated_text': 'The earth is not flat. It is an endless'}]

## 7.4 Few-shot learning

In [131]:
print(generator("""Sentiment Analysis
Text: I hate it when my phone battery dies.
Sentiment: Negative
###
Text: My day has been really great!
Sentiment: Positive
###
Text: Not a fan when it is cloudy
Sentiment:""", top_k=2, temperature=0.1, max_length=55)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sentiment Analysis
Text: I hate it when my phone battery dies.
Sentiment: Negative
###
Text: My day has been really great!
Sentiment: Positive
###
Text: Not a fan when it is cloudy
Sentiment: Negative



In [132]:
print(generator("""Question/Answering
C: Google was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control 56 percent of the stockholder voting power through supervoting stock.
Q: When was Google founded?
A: 1998
###
C: Hugging Face is a company which develops social AI-run chatbot applications. It was established in 2016 by Clement Delangue and Julien Chaumond. The company is based in Brooklyn, New York, United States.
Q: What does Hugging Face develop?
A: social AI-run chatbot applications
###
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: What division do the Jets play in?
A:""", top_k=2, beams=2, max_length=215, temperature=0.5)[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question/Answering
C: Google was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control 56 percent of the stockholder voting power through supervoting stock.
Q: When was Google founded?
A: 1998
###
C: Hugging Face is a company which develops social AI-run chatbot applications. It was established in 2016 by Clement Delangue and Julien Chaumond. The company is based in Brooklyn, New York, United States.
Q: What does Hugging Face develop?
A: social AI-run chatbot applications
###
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: What division do the Jets play in?
A: American Football Conference (AFC) East


In [96]:
## Zero Shot Learning

In [133]:
# Same question as before, with no previous examples ie Zero-shot learning. Still works
print(generator(
    '''Question/Answering
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: What division do the Jets play in?
A:''',
    top_k=2, beams=2, max_length=80, temperature=0.5)[0]['generated_text']
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question/Answering
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: What division do the Jets play in?
A: The New York Jets play in the American Football Conference East


In [98]:
# Zero-shot doesn't work as much with the sentiment analysis example
print(generator("""Sentiment Analysis
Text: This new music video was so good
Sentiment:""", top_k=2, temperature=0.1, max_length=55)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sentiment Analysis
Text: This new music video was so good
Sentiment: I love it
Sentiment: I love it
Sentiment: I love it
Sentiment: I love it
Sentiment: I love it
Sentiment: I love it


In [99]:
# Zero-shot abstractive summarization

In [100]:
to_summarize = """This training will focus on how the GPT family of models are used for NLP tasks including abstractive text summarization and natural language generation. The training will begin with an introduction to necessary concepts including masked self attention, language models, and transformers and then build on those concepts to introduce the GPT architecture. We will then move into how GPT is used for multiple natural language processing tasks with hands-on examples of using pre-trained GPT-2 models as well as fine-tuning these models on custom corpora.

GPT models are some of the most relevant NLP architectures today and it is closely related to other important NLP deep learning models like BERT. Both of these models are derived from the newly invented transformer architecture and represent an inflection point in how machines process language and context.

The Natural Language Processing with Next-Generation Transformer Architectures series of online trainings provides a comprehensive overview of state-of-the-art natural language processing (NLP) models including GPT and BERT which are derived from the modern attention-driven transformer architecture and the applications these models are used to solve today. All of the trainings in the series blend theory and application through the combination of visual mathematical explanations, straightforward applicable Python examples within hands-on Jupyter notebook demos, and comprehensive case studies featuring modern problems solvable by NLP models. (Note that at any given time, only a subset of these classes will be scheduled and open for registration.)"""

In [134]:
print(generator(
    f"""Summarization Task:\n{to_summarize}\nTL;DR:""", 
    max_length=512, top_k=5, beams=5, temperature=0.5, no_repeat_ngram_size=2
)[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Summarization Task:
This training will focus on how the GPT family of models are used for NLP tasks including abstractive text summarization and natural language generation. The training will begin with an introduction to necessary concepts including masked self attention, language models, and transformers and then build on those concepts to introduce the GPT architecture. We will then move into how GPT is used for multiple natural language processing tasks with hands-on examples of using pre-trained GPT-2 models as well as fine-tuning these models on custom corpora.

GPT models are some of the most relevant NLP architectures today and it is closely related to other important NLP deep learning models like BERT. Both of these models are derived from the newly invented transformer architecture and represent an inflection point in how machines process language and context.

The Natural Language Processing with Next-Generation Transformer Architectures series of online trainings provides a